# Objective: To estimate the revenue figure of Uber in a Year in NY and it's growth and also aimed to expose all the interesting insights that can be derived from a detailed analysis of the dataset.

In [27]:
#Importing the relevent liberaries and defining the settings for plotting.
%matplotlib inline
import sys
print(sys.version)

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
import time

sns.set()
pal = sns.hls_palette(10, h = 0.5)
sns.set_palette(pal)

#Avoiding the display of scientific notation and show precesion upto 4 decimal places.
pd.set_option("display.float_format", lambda x: "%.4f"%x)

3.9.7 (default, Sep 16 2021, 16:59:28) [MSC v.1916 64 bit (AMD64)]


In [28]:
df_uber = pd.read_csv("Uber_nyc_data.csv")


This uber dataset contains data about Uber's ridership between September 2014 and August 2015. This dataset contains ~31 million rows and columns of origin, destination, pickup date and time, trip distance, and duration.

The combination of trip distance and duration allows for estimation of UBER's revenue for each trip in NYC. On another hand, the pickup and the drop-off location has been anonymized and groped as taxi-zones instead of geographical co-ordinates.This is better attempt to preserve the data privacy.

In [29]:
df_uber.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30925738 entries, 0 to 30925737
Data columns (total 6 columns):
 #   Column           Dtype  
---  ------           -----  
 0   id               int64  
 1   origin_taz       object 
 2   destination_taz  object 
 3   pickup_datetime  object 
 4   trip_distance    float64
 5   trip_duration    object 
dtypes: float64(1), int64(1), object(4)
memory usage: 1.4+ GB


In [30]:
df_uber.head()

,id,origin_taz,destination_taz,pickup_datetime,trip_distance,trip_duration
0,252581,7C,6A,2014-09-01 09:00:00,4.2500,0:15:11
1,252582,7B,15,2014-09-01 18:00:00,10.1700,0:34:05
2,252583,11,2A,2014-09-01 17:00:00,4.0200,0:17:06
3,252584,3B,4A,2014-09-01 13:00:00,1.4600,0:06:32
4,252585,2A,10,2014-09-01 14:00:00,8.3100,0:26:17


In [31]:
len(df_uber.id.unique())

30925738

# Check for Missing or Possibly Erroreous data.

In [32]:
len(df_uber[df_uber.duplicated()==True])

0

No duplicate entries are present in the dataset.

In [33]:
df_uber.isnull().sum()

id                       0
origin_taz               0
destination_taz    1273023
pickup_datetime          0
trip_distance           38
trip_duration           38
dtype: int64

1. Show total null values per column.
2. About 1.3 million data of destination_taz is missing, 38 rows misses of trip distance and trip duration.

In [34]:
arr1 = df_uber.origin_taz.unique()
print(arr1)
len(arr1)

['7C' '7B' '11' '3B' '2A' '5B' '10' '2B' '9' '6B' '15' '4C' '5A' '8' '14'
 '4A' '7A' '1' '16' '3A' '2C' '6A' '12' '13' '3C' '4B' '5C' '17']


28

28 pickup locations were there in the NYC.

In [35]:
arr2 = df_uber.destination_taz.unique()
print(arr2)
len(arr2)

['6A' '15' '2A' '4A' '10' '4C' '7A' '3C' '5B' '14' '8' nan '7C' '12' '2C'
 '1' '6B' '5C' '9' '3A' '2B' '11' '7B' '5A' '13' '4B' '18' '16' '3B' '17']


30

30 Unique destination code i.e. drop location is same as origin except for 'nan' and '18'.

In [36]:
set(arr2) - set(arr1)

{'18', nan}

Expectation for 18?

18 may represents **EWR airport?**(EWR airport or Newark Airport is a airport near New York City (~25 km/15 miles) and many of its passengers are flying to or from NYC, that's why it's considered one of NYC's 3 regional airports (along with LaGuardia and Kennedy.)

In [37]:
df_uber[df_uber.destination_taz.isnull()].head()

,id,origin_taz,destination_taz,pickup_datetime,trip_distance,trip_duration
15,252596,6B,NaN,2014-09-01 17:00:00,11.6700,0:23:45
32,252613,9,NaN,2014-09-01 16:00:00,80.6900,1:59:37
63,252644,9,NaN,2014-09-01 18:00:00,23.0700,1:03:37
141,252722,9,NaN,2014-09-01 10:00:00,21.7300,0:41:12
148,252729,4B,NaN,2014-09-01 10:00:00,4.6200,0:14:55


* Checked some of the data with the missing data. One idea is to fill the missing values by prediction based on the other features.

In [38]:
df38 = df_uber[df_uber.trip_duration.isnull() & df_uber.trip_distance.isnull()]
df38.head()

,id,origin_taz,destination_taz,pickup_datetime,trip_distance,trip_duration
15155317,39535,2A,11,2015-04-25 12:00:00,NaN,NaN
15245057,1009076,2A,2A,2015-04-26 01:00:00,NaN,NaN
16519652,15028665,2A,7C,2015-04-29 21:00:00,NaN,NaN
17148253,22250173,2A,2A,2015-04-12 02:00:00,NaN,NaN
17297563,23716998,2C,11,2015-04-25 13:00:00,NaN,NaN


* On investigating, we found that all 38 rows have missing trip_distance and trip_duration but origin and destination is known which implies the fare has been taken once the trip is done and thus revenue generated.

In [39]:
# All cases when trip duration is null and trip distance is not null.
len(df_uber[(df_uber.trip_duration.isnull()==False) & (df_uber.trip_distance.isnull())])

0

# Parse the pickup_datetime to datetime type and create related features.

In [41]:
#None of the pickup time data offer mintues or seconds precision
len(df_uber[df_uber.pickup_datetime.apply(lambda x: x[-5:] == '00:00')==False])

0

* Since the pickup_datetime is string data type so we will use date_parser to method to convert it into date time object.

In [42]:
# Function used to convert string into columns into an array of datetime instances.
def dateparser(s):
    return datetime.datetime(int(s[0:4]),int(s[5:7]), int(s[8:10]),int(s[11:13]))

* date_parser Function takes a string in the format of yyyy-mm-dd hh:mm:ss and returns the same as the datetime object. 

In [43]:
df_uber.head()

,id,origin_taz,destination_taz,pickup_datetime,trip_distance,trip_duration
0,252581,7C,6A,2014-09-01 09:00:00,4.2500,0:15:11
1,252582,7B,15,2014-09-01 18:00:00,10.1700,0:34:05
2,252583,11,2A,2014-09-01 17:00:00,4.0200,0:17:06
3,252584,3B,4A,2014-09-01 13:00:00,1.4600,0:06:32
4,252585,2A,10,2014-09-01 14:00:00,8.3100,0:26:17


In [46]:
# Apply parser to data
t0 = time.time()
df_uber['pu_date_hour'] = df_uber.pickup_datetime.apply(dateparser)
time.time() - t0

32.465770959854126

In [47]:
df_uber.head()

,id,origin_taz,destination_taz,pickup_datetime,trip_distance,trip_duration,pu_date_hour
0,252581,7C,6A,2014-09-01 09:00:00,4.2500,0:15:11,2014-09-01 09:00:00
1,252582,7B,15,2014-09-01 18:00:00,10.1700,0:34:05,2014-09-01 18:00:00
2,252583,11,2A,2014-09-01 17:00:00,4.0200,0:17:06,2014-09-01 17:00:00
3,252584,3B,4A,2014-09-01 13:00:00,1.4600,0:06:32,2014-09-01 13:00:00
4,252585,2A,10,2014-09-01 14:00:00,8.3100,0:26:17,2014-09-01 14:00:00


* Now we can drop pickup_datetime column

In [51]:
df_uber = df_uber.drop('pickup_datetime', axis=1)

In [52]:
df_uber.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30925738 entries, 0 to 30925737
Data columns (total 6 columns):
 #   Column           Dtype         
---  ------           -----         
 0   id               int64         
 1   origin_taz       object        
 2   destination_taz  object        
 3   trip_distance    float64       
 4   trip_duration    object        
 5   pu_date_hour     datetime64[ns]
dtypes: datetime64[ns](1), float64(1), int64(1), object(3)
memory usage: 1.4+ GB


* Now the pu_date_hour is of the format datetime

In [53]:
beginning = df_uber.pu_date_hour.min()
end =  df_uber.pu_date_hour.max()
print(beginning,end,beginning - end)

2014-09-01 00:00:00 2015-09-01 00:00:00 -365 days +00:00:00


* The dataset contains the details of exactly **365 days** from **09th Sep 2014** to **09th Sep 2015**.

# Getting the federal holidays for the period(very useful in analysis of uber revenue).

In [57]:
# Getting all federal holidays in a period.
from pandas.tseries.holiday import USFederalHolidayCalendar

holidays = USFederalHolidayCalendar().holidays(beginning,end, return_name = True)
holidays

2014-09-01                     Labor Day
2014-10-13                  Columbus Day
2014-11-11                  Veterans Day
2014-11-27                  Thanksgiving
2014-12-25                     Christmas
2015-01-01                 New Years Day
2015-01-19    Martin Luther King Jr. Day
2015-02-16                Presidents Day
2015-05-25                  Memorial Day
2015-07-03                      July 4th
dtype: object

* We get the list of holidays occurs during the same time period of 365 days and we analyze the no. of uber rides a day before holiday and a day after holiday and also on the holiday

In [58]:
#Extract the weekday for each holiday
holidays.index.map(lambda x: x.strftime('%a'))

Index(['Mon', 'Mon', 'Tue', 'Thu', 'Thu', 'Thu', 'Mon', 'Mon', 'Mon', 'Fri'], dtype='object')

In [61]:
# Let's add the day before and day after the holiday
holiday_all = pd.concat([holidays,"Day After" + holidays.shift(1,'D'), "Day Before" + holidays.shift(-1,'D')])
holiday_all = holiday_all.sort_index()
holiday_all.head(10)

2014-08-31       Day BeforeLabor Day
2014-09-01                 Labor Day
2014-09-02        Day AfterLabor Day
2014-10-12    Day BeforeColumbus Day
2014-10-13              Columbus Day
2014-10-14     Day AfterColumbus Day
2014-11-10    Day BeforeVeterans Day
2014-11-11              Veterans Day
2014-11-12     Day AfterVeterans Day
2014-11-26    Day BeforeThanksgiving
dtype: object

# Feature Engineering

**Creating other date and time features**

In [68]:
# Creating a column date without the time.
df_uber['pu_date'] = pd.Series(map(lambda x: x.astype('datetime64[D]'),df_uber['pu_date_hour'].values))


In [69]:
df_uber.head()

,id,origin_taz,destination_taz,trip_distance,trip_duration,pu_date_hour,pu_date
0,252581,7C,6A,4.2500,0:15:11,2014-09-01 09:00:00,2014-09-01
1,252582,7B,15,10.1700,0:34:05,2014-09-01 18:00:00,2014-09-01
2,252583,11,2A,4.0200,0:17:06,2014-09-01 17:00:00,2014-09-01
3,252584,3B,4A,1.4600,0:06:32,2014-09-01 13:00:00,2014-09-01
4,252585,2A,10,8.3100,0:26:17,2014-09-01 14:00:00,2014-09-01


In [70]:
# Getting month and year from the pickup timestamp
df_uber['year'] = df_uber['pu_date_hour'].dt.year
df_uber['month'] = df_uber['pu_date_hour'].dt.month

In [71]:
df_uber.head()

,id,origin_taz,destination_taz,trip_distance,trip_duration,pu_date_hour,pu_date,year,month
0,252581,7C,6A,4.2500,0:15:11,2014-09-01 09:00:00,2014-09-01,2014,9
1,252582,7B,15,10.1700,0:34:05,2014-09-01 18:00:00,2014-09-01,2014,9
2,252583,11,2A,4.0200,0:17:06,2014-09-01 17:00:00,2014-09-01,2014,9
3,252584,3B,4A,1.4600,0:06:32,2014-09-01 13:00:00,2014-09-01,2014,9
4,252585,2A,10,8.3100,0:26:17,2014-09-01 14:00:00,2014-09-01,2014,9


In [72]:
# Getting trip pickup day of the month.
df_uber['day'] = df_uber['pu_date_hour'].dt.day

In [73]:
df_uber.head()

,id,origin_taz,destination_taz,trip_distance,trip_duration,pu_date_hour,pu_date,year,month,day
0,252581,7C,6A,4.2500,0:15:11,2014-09-01 09:00:00,2014-09-01,2014,9,1
1,252582,7B,15,10.1700,0:34:05,2014-09-01 18:00:00,2014-09-01,2014,9,1
2,252583,11,2A,4.0200,0:17:06,2014-09-01 17:00:00,2014-09-01,2014,9,1
3,252584,3B,4A,1.4600,0:06:32,2014-09-01 13:00:00,2014-09-01,2014,9,1
4,252585,2A,10,8.3100,0:26:17,2014-09-01 14:00:00,2014-09-01,2014,9,1


In [74]:
# Get trip pickup hour from timestamp
df_uber['hour'] = df_uber['pu_date_hour'].dt.hour

In [75]:
# Get the weekdays of pickup
df_uber["weekday"]= df_uber['pu_date_hour'].dt.dayofweek

In [76]:
df_uber.head()

,id,origin_taz,destination_taz,trip_distance,trip_duration,pu_date_hour,pu_date,year,month,day,hour,weekday
0,252581,7C,6A,4.2500,0:15:11,2014-09-01 09:00:00,2014-09-01,2014,9,1,9,0
1,252582,7B,15,10.1700,0:34:05,2014-09-01 18:00:00,2014-09-01,2014,9,1,18,0
2,252583,11,2A,4.0200,0:17:06,2014-09-01 17:00:00,2014-09-01,2014,9,1,17,0
3,252584,3B,4A,1.4600,0:06:32,2014-09-01 13:00:00,2014-09-01,2014,9,1,13,0
4,252585,2A,10,8.3100,0:26:17,2014-09-01 14:00:00,2014-09-01,2014,9,1,14,0


In [77]:
df_uber.weekday.value_counts()

5    5142832
4    4805002
3    4625511
6    4331195
2    4301537
1    4010434
0    3709227
Name: weekday, dtype: int64

* Here 0 --> Monday and 6-->Sunday

**Final value after Feature Enginnering on datetime**

In [78]:
df_uber.head()

,id,origin_taz,destination_taz,trip_distance,trip_duration,pu_date_hour,pu_date,year,month,day,hour,weekday
0,252581,7C,6A,4.2500,0:15:11,2014-09-01 09:00:00,2014-09-01,2014,9,1,9,0
1,252582,7B,15,10.1700,0:34:05,2014-09-01 18:00:00,2014-09-01,2014,9,1,18,0
2,252583,11,2A,4.0200,0:17:06,2014-09-01 17:00:00,2014-09-01,2014,9,1,17,0
3,252584,3B,4A,1.4600,0:06:32,2014-09-01 13:00:00,2014-09-01,2014,9,1,13,0
4,252585,2A,10,8.3100,0:26:17,2014-09-01 14:00:00,2014-09-01,2014,9,1,14,0


In [79]:
df_uber.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30925738 entries, 0 to 30925737
Data columns (total 12 columns):
 #   Column           Dtype         
---  ------           -----         
 0   id               int64         
 1   origin_taz       object        
 2   destination_taz  object        
 3   trip_distance    float64       
 4   trip_duration    object        
 5   pu_date_hour     datetime64[ns]
 6   pu_date          datetime64[ns]
 7   year             int64         
 8   month            int64         
 9   day              int64         
 10  hour             int64         
 11  weekday          int64         
dtypes: datetime64[ns](2), float64(1), int64(6), object(3)
memory usage: 2.8+ GB


# Tranformation of Trip Duration data.

In [80]:
uniq_dur = df_uber[df_uber.trip_duration.isnull()==False].trip_duration.unique()

In [82]:
print(len(uniq_dur))

19393


In [85]:
# Among uique duration strings, find how many represent>= 10 hours of durationnnnnnn
long_duration = []
for item in uniq_dur:
    if len(item) != 7:
        long_duration.append(item)
        
# Long duration
print(len(long_duration))

386
